In [ ]:
#join trades and quotes
import onetick.query as otq
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
pd.set_option('display.width', 200)

In [ ]:
symbol='FULL_DEMO_L1::GS'
start=datetime(2005,1,3,9,30)
end = datetime(2005,1,3,16)

trds = otq.Passthrough(fields='PRICE')
qtes = otq.Passthrough(fields='BID_PRICE,ASK_PRICE')
num_ticks = otq.NumTicks(is_running_aggr=True,output_field_name='TICK_ID')

join = otq.JoinByTime(leading_sources='t')
trds >> num_ticks.tick_type('TRD').node_name('t') >> join << num_ticks.copy().tick_type('QTE').node_name('q') << qtes

graph = otq.Graph(join)

# render graph
graph.render()

data = otq.run(graph,symbol,start,end)
df = pd.DataFrame(data[symbol])
df.set_index('Time',inplace=True)
df[['t.PRICE']].plot(figsize=(12,7))
plt.show()
df.tail()

In [ ]:
symbol='FULL_DEMO_L1::GS'
start=datetime(2005,1,3,9,30)
end = datetime(2005,1,3,16)

trds = otq.Passthrough(fields='PRICE')
qtes = otq.Passthrough(fields='BID_PRICE,ASK_PRICE')
num_ticks = otq.NumTicks(is_running_aggr=True,output_field_name='TICK_ID')

join = otq.JoinByTime(leading_sources='t')
trds = otq.Chainlet(trds,num_ticks.copy().tick_type('TRD').node_name('t'))
qtes = otq.Chainlet(qtes,num_ticks.copy().tick_type('QTE').node_name('q'))
join = otq.attach([trds,qtes],otq.JoinByTime(leading_sources='t'))

graph = otq.Graph(join)

# render graph
graph.render()

data = otq.run(graph,symbol,start,end)
df = pd.DataFrame(data[symbol])
df.set_index('Time',inplace=True)
df[['q.ASK_PRICE']].plot(figsize=(12,7))
plt.show()
df.tail()


In [ ]:
#pair correlation
sym1 = 'FULL_DEMO_L1::MSFT'
sym2 = 'FULL_DEMO_L1::CSCO'

def printg(graph, sym1=sym1, sym2=sym2):
    import collections
    
    results = otq.run(graph, symbols=None, start=datetime(2005,1,3,9,30), end=datetime(2005,1,3,16))
    df = pd.DataFrame(results[''])
    pd.set_option('display.width', 200)
    print(df.head())
    df[['CORRELATION']].plot(title='Correlation {}-{}'.format(sym1,sym2),figsize=(12,7))


#params for Compute EP object
#===========================================================================
compute_string='FIRST(TIME_SERIES_TYPE=STATE_TS) OPEN, LAST(TIME_SERIES_TYPE=STATE_TS) CLOSE'

#build graph
trd_pass = otq.Passthrough(fields='PRICE',go_back_to_first_tick=86400).tick_type('TRD').symbol(sym1)
compute = otq.ComputeEp(compute=compute_string, bucket_interval=0,output_interval=60,\
                      is_running_aggr=False,append_output_field_name=False) 
add_field = otq.AddField(field='RETURN double', value='(CLOSE-CLOSE[-1])/CLOSE[-1]')
symbol_pair = otq.AddField(field='SYMBOL_PAIR',value='"{}-{}"'.format(sym1,sym2))

#build graph
root = trd_pass
join = otq.JoinByTime(source_order='L,R', match_if_identical_times=True)
trd_pass >> compute >> add_field.node_name('L') >> join << add_field.copy().node_name('R') << compute.copy() << trd_pass.copy().symbol(sym2)
join >> otq.Correlation(is_running_aggr=True,input_field1_name='L.RETURN', input_field2_name='R.RETURN',output_field_name='CORRELATION') \
      >> symbol_pair

g = otq.Graph(root)
g.render()
g.save_to_file('C:/temp/pair_corr.otq', 'dummy', symbols='', start=datetime(2005,1,3),end=datetime(2005,1,3))
printg(g)
plt.show() 

In [ ]:
symbols = ['FULL_DEMO_L1::A',
           'FULL_DEMO_L1::AA',
           'FULL_DEMO_L1::AAA',
           'FULL_DEMO_L1::C',
           'FULL_DEMO_L1::CSCO',
           'FULL_DEMO_L1::CVX',
           'FULL_DEMO_L1::GS',
           'FULL_DEMO_L1::IBM',
           'FULL_DEMO_L1::LEH',
           'FULL_DEMO_L1::MS',
           'FULL_DEMO_L1::MSFT',
           'FULL_DEMO_L1::ORCL',
           'FULL_DEMO_L1::VZ',
           'FULL_DEMO_L1::XRX']
start_time, end_time = datetime(2006, 6, 1, 9, 30), datetime(2006, 6, 1, 16)
num_of_top_ticks = 10

# create graph with EPs
merge = otq.Merge()
join = otq.JoinByTime(leading_sources='H',match_if_identical_times=True,same_timestamp_join_policy='EACH_FOR_LEADER_WITH_LATEST')

chain = otq.Chainlet(otq.Passthrough(fields='SIZE').tick_type('TRD'),
                    otq.Sum(output_field_name='VOLUME',input_field_name='SIZE'),
                    otq.Presort().symbols(symbols),
                    merge,
                    #otq.HighTick(num_ticks=num_of_top_ticks,input_field_name='VOLUME').node_name('H'),
                    )
head = chain >> otq.HighTick(num_ticks=num_of_top_ticks,input_field_name='VOLUME').node_name('H')
ssum = chain >> otq.Sum(output_field_name='VOLUME',input_field_name='VOLUME').node_name('T')
head >> join << ssum
#chain >> join << otq.Sum(output_field_name='VOLUME',input_field_name='VOLUME').node_name('T') << merge
join >> otq.AddField(field='PERC_OF_TOTAL',value='H.VOLUME/T.VOLUME') 

graph = otq.Graph(chain)
graph.render()


In [ ]:
data = otq.run(graph,symbols=None,start=start_time,end=end_time)
df = pd.DataFrame(data[''])

df.set_index(['H.SYMBOL_NAME'],inplace=True)

#plot
ax = df['PERC_OF_TOTAL'].plot(kind='bar',title='Percent of Total',figsize=(15,10),width=1)
ax.set_ylabel('Percent(%)')
ax.set_xlabel('Symbol')
ax.axhline(y=0, xmin=0, color='black')

plt.show()
df.head()